In [1]:
# imports
import pandas as pd
import numpy as np
import math
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
# test variables 
# test variables 
ORM = 42
reps = [5, 5, 5, 3, 3, 3]
p1rm = [45, 50, 55, 65, 75, 85]
lift = 'Bench'

program_data = {
    'lift': lift,
    '1RM': ORM,
    'reps': reps,
    '%1RM':p1rm
}

plates_all_list = [0.25, 0.5, 1.25, 2.5, 5, 10, 15, 20, 25]
plates_quantity = [4, 4, 6, 6, 10, 10, 10, 8, 8]

plate_data = {'plate': plates_all_list, 
              'available': plates_quantity}

# standard bar weight = 20kg
bar = 20

In [3]:
# time to use some *good programming practices* and shove all this shit 
# into fucntions

# generate initial dataframes
# split out into seperate functions since the plates will stay the same but can have mulitple different programs

def create_program_df(program_data):
    # get val for lift ORM for calculating weight
    ORM_calc = program_data.get('1RM')
    # create dataframe
    df = pd.DataFrame(program_data)
    df['weight'] = df['%1RM'].apply(lambda x: x/100*ORM_calc)  
    return df

def create_plates_df(plate_data):
    # create dataframe
    plates = pd.DataFrame(plate_data)
    
    # half the available plates for calculation value, round to deal with uneven
    plates['avail_calc'] = (plates['available'].apply(lambda x: x/2)).round(0)

    # create list of available plates repeating per quantity to use in calculation function
    plates_list = list(plates['plate'].reindex(plates.index.repeat(plates.avail_calc)))
    return plates, plates_list

program = create_program_df(program_data)
plates, plates_list = create_plates_df(plate_data)

program

,lift,1RM,reps,%1RM,weight
0,Bench,42,5,45,18.9
1,Bench,42,5,50,21.0
2,Bench,42,5,55,23.1
3,Bench,42,3,65,27.3
4,Bench,42,3,75,31.5
5,Bench,42,3,85,35.7


In [4]:
# generator from stackoverflow that does the job
# numbers = plates list
# target = weight
def subset_sum(numbers, target, partial=[], partial_sum=0):
    if partial_sum == target:
        yield partial
    if partial_sum >= target:
        return
    for i, n in enumerate(numbers):
        remaining = numbers[i + 1:]
        yield from subset_sum(remaining, target, partial + [n], partial_sum + n)

In [5]:
# get list of weights from program dataframe, round from float to int
list_weights = [(math.ceil(x*2)/2) for x in list(program['weight'])]
# to get the amount needed per side, remove the bar weight and half the total
list_weights = [(x-bar)/2 for x in list_weights]

# create dictionary and call generator to populate using weight as key
d = {}
for i, val in enumerate(list_weights):
    newlist = list(subset_sum(plates_list, val))
    key = val
    d[key] = newlist

# build dataframe from dictionary
df = pd.DataFrame(list(d.items()),columns=['weight','combinations'])
# explode to get each combination per row
df = df.explode('combinations').set_index(['weight'], append=True)

# deal with negative values (if weight is less than the bar)
# split dataframe into 0 values
df_na = df[(df['combinations'].isna()) | (df["combinations"].str.len() == 0)]
df_na = df_na.reset_index(level=1)
# assign string based on weight
df_na['combinations'] = np.where(
    df_na['weight'] == 0, 'bar weight only', np.where(
    df_na['weight'] <  0, 'weight under bar weight', 'err')) 

# non-na df
df = df[(df['combinations'].notna())]
df = df[df['combinations'].map(lambda d: len(d)) > 0]

# convert lists to strings seperated by ;
df['combinations'] = ['; '.join(map(str, l)) for l in df['combinations']]

# drop duplicates & remove the multi-index
df = df.drop_duplicates()

df = df.reset_index(level=1)

# rejoin df_na to df now that combinations is a string column
df = pd.concat([df, df_na], axis=0)

# narrow down dataframe
# get best combination (i.e. lowest number of elements)
df['len'] = df['combinations'].str.count(';')


# sort into top 5 options 
df = df.sort_values('len', ascending=True).groupby(['weight']).head(5)
df = df.sort_values(['weight','len']).drop(['len'], axis=1)

# join back to original dataframe
# recalculate the required weight to join back 
final = df.rename(columns={'weight':'plate_weight','combinations':'plates' })

# add bar weight and * 2
final['weight'] = final['plate_weight'].apply(lambda x: (x*2)+bar)

# merge onto program dataframe
# round up column for join
program.rename(columns={'weight':'unrounded weight'}, inplace=True)
program['weight'] = program['unrounded weight'].apply(lambda x: (math.ceil(x*2)/2))  
#merge
final = program.merge(final, on='weight', how='left')
final.sort_values(['reps','%1RM','weight'], ascending=False)

# cleanup 
final.rename(columns={'unrounded weight':'weight', 'weight':'weight (rounded)', 'plate_weight':'plate weight'})

final
#program


,lift,1RM,reps,%1RM,unrounded weight,weight,plate_weight,plates
0,Bench,42,5,45,18.9,19.0,-0.50,weight under bar weight
1,Bench,42,5,50,21.0,21.0,0.50,0.5
2,Bench,42,5,50,21.0,21.0,0.50,0.25; 0.25
3,Bench,42,5,55,23.1,23.5,1.75,0.5; 1.25
4,Bench,42,5,55,23.1,23.5,1.75,0.25; 0.25; 1.25
5,Bench,42,3,65,27.3,27.5,3.75,1.25; 2.5
6,Bench,42,3,65,27.3,27.5,3.75,1.25; 1.25; 1.25
7,Bench,42,3,65,27.3,27.5,3.75,0.25; 0.5; 0.5; 2.5
8,Bench,42,3,65,27.3,27.5,3.75,0.25; 0.5; 0.5; 1.25; 1.25
9,Bench,42,3,75,31.5,31.5,5.75,0.25; 0.5; 5.0


In [6]:
final.columns

Index(['lift', '1RM', 'reps', '%1RM', 'unrounded weight', 'weight',
       'plate_weight', 'plates'],
      dtype='object')

In [7]:
final_dict = final.to_dict(orient='records')


#final_dict = {val1: val2.to_dict(orient='records') for val1,val2 in final.groupby("%1RM")}

final_dict

[{'lift': 'Bench',
  '1RM': 42,
  'reps': 5,
  '%1RM': 45,
  'unrounded weight': 18.900000000000002,
  'weight': 19.0,
  'plate_weight': -0.5,
  'plates': 'weight under bar weight'},
 {'lift': 'Bench',
  '1RM': 42,
  'reps': 5,
  '%1RM': 50,
  'unrounded weight': 21.0,
  'weight': 21.0,
  'plate_weight': 0.5,
  'plates': '0.5'},
 {'lift': 'Bench',
  '1RM': 42,
  'reps': 5,
  '%1RM': 50,
  'unrounded weight': 21.0,
  'weight': 21.0,
  'plate_weight': 0.5,
  'plates': '0.25; 0.25'},
 {'lift': 'Bench',
  '1RM': 42,
  'reps': 5,
  '%1RM': 55,
  'unrounded weight': 23.1,
  'weight': 23.5,
  'plate_weight': 1.75,
  'plates': '0.5; 1.25'},
 {'lift': 'Bench',
  '1RM': 42,
  'reps': 5,
  '%1RM': 55,
  'unrounded weight': 23.1,
  'weight': 23.5,
  'plate_weight': 1.75,
  'plates': '0.25; 0.25; 1.25'},
 {'lift': 'Bench',
  '1RM': 42,
  'reps': 3,
  '%1RM': 65,
  'unrounded weight': 27.3,
  'weight': 27.5,
  'plate_weight': 3.75,
  'plates': '1.25; 2.5'},
 {'lift': 'Bench',
  '1RM': 42,
  'reps': 

In [51]:
output_series = final.groupby(['lift', '1RM', '%1RM', 'reps', 'unrounded weight', 'weight',
       'plate_weight'])['plates'].apply(list)

output_series

new = final.groupby(['lift', '1RM', '%1RM', 'reps', 'unrounded weight', 'weight',
       'plate_weight'], as_index = False).agg({'plates': ' | '.join})

new

from IPython.display import display, HTML
def pretty_print(df):
    return display( HTML( df.to_html().replace("|","<br>---------<br>") ) )

pretty_print(new)

,lift,1RM,%1RM,reps,unrounded weight,weight,plate_weight,plates
0,Bench,42,45,5,18.9,19.0,-0.50,weight under bar weight
1,Bench,42,50,5,21.0,21.0,0.50,0.5 --------- 0.25; 0.25
2,Bench,42,55,5,23.1,23.5,1.75,0.5; 1.25 --------- 0.25; 0.25; 1.25
3,Bench,42,65,3,27.3,27.5,3.75,1.25; 2.5 --------- 1.25; 1.25; 1.25 --------- 0.25; 0.5; 0.5; 2.5 --------- 0.25; 0.5; 0.5; 1.25; 1.25
4,Bench,42,75,3,31.5,31.5,5.75,0.25; 0.5; 5.0 --------- 0.25; 0.5; 2.5; 2.5 --------- 0.25; 0.5; 1.25; 1.25; 2.5
5,Bench,42,85,3,35.7,36.0,8.00,0.5; 2.5; 5.0 --------- 0.25; 0.25; 2.5; 5.0 --------- 0.5; 1.25; 1.25; 5.0 --------- 0.5; 2.5; 2.5; 2.5 --------- 0.25; 0.25; 1.25; 1.25; 5.0


In [46]:
new_dict = new.to_dict(orient = 'index')

new_dict

{0: {'lift': 'Bench',
  '1RM': 42,
  '%1RM': 45,
  'reps': 5,
  'unrounded weight': 18.900000000000002,
  'weight': 19.0,
  'plate_weight': -0.5,
  'plates': 'weight under bar weight'},
 1: {'lift': 'Bench',
  '1RM': 42,
  '%1RM': 50,
  'reps': 5,
  'unrounded weight': 21.0,
  'weight': 21.0,
  'plate_weight': 0.5,
  'plates': '0.5 | 0.25; 0.25'},
 2: {'lift': 'Bench',
  '1RM': 42,
  '%1RM': 55,
  'reps': 5,
  'unrounded weight': 23.1,
  'weight': 23.5,
  'plate_weight': 1.75,
  'plates': '0.5; 1.25 | 0.25; 0.25; 1.25'},
 3: {'lift': 'Bench',
  '1RM': 42,
  '%1RM': 65,
  'reps': 3,
  'unrounded weight': 27.3,
  'weight': 27.5,
  'plate_weight': 3.75,
  'plates': '1.25; 2.5 | 1.25; 1.25; 1.25 | 0.25; 0.5; 0.5; 2.5 | 0.25; 0.5; 0.5; 1.25; 1.25'},
 4: {'lift': 'Bench',
  '1RM': 42,
  '%1RM': 75,
  'reps': 3,
  'unrounded weight': 31.5,
  'weight': 31.5,
  'plate_weight': 5.75,
  'plates': '0.25; 0.5; 5.0 | 0.25; 0.5; 2.5; 2.5 | 0.25; 0.5; 1.25; 1.25; 2.5'},
 5: {'lift': 'Bench',
  '1RM': 

{'lift': 'Bench',
 '1RM': 42,
 '%1RM': 45,
 'reps': 5,
 'unrounded weight': 18.900000000000002,
 'weight': 19.0,
 'plate_weight': -0.5,
 'plates': 'weight under bar weight'}

In [47]:
# basic print
template = '''Lift: {lift} | 1RM: {1RM}
\n
% 1RM x reps x actual weight
{%1RM} x {reps} x {weight}
\n\n
Bar weight: {bar}
Plates weight: {plate_weight}
\n
Plate combinations:
\n
{plates}
'''

for item in new_dict:
    print(template.format(**new_dict))

KeyError: 'lift'

In [ ]:
'''
# create dataframe with the plate combinations, narrow down to top 5 per weight based on least number of plates needed
def generate_combinations(program_df, plates_list, bar):
    # get list of weights from program dataframe, round from float to int
    list_weights = [round(x) for x in list(program['weight'])]
    # to get the amount needed per side, remove the bar weight and half the total
    list_weights = [(x-bar)/2 for x in list_weights]

    # create dictionary and call generator to populate using weight as key
    d = {}
    for i, val in enumerate(list_weights):
        newlist = list(subset_sum(plates_list, val))
        key = val
        d[key] = newlist 

    # build dataframe from dictionary
    df = pd.DataFrame(list(d.items()),columns=['weight','combinations'])
    # explode to get each combination per row
    df = df.explode('combinations').set_index(['weight'], append=True)

    # deal with negative values (if weight is less than the bar)
    # split dataframe into 0 values
    df_na = df[(df['combinations'].isna()) | (df["combinations"].str.len() == 0)]
    df_na = df_na.reset_index(level=1)
    # assign string based on weight
    df_na['combinations'] = np.where(
        df_na['weight'] == 0, 'bar weight only', np.where(
        df_na['weight'] <  0, 'weight under bar weight', 'err')) 

    # non-na df
    df = df[(df['combinations'].notna())]
    df = df[df['combinations'].map(lambda d: len(d)) > 0]

    # convert lists to strings seperated by ;
    df['combinations'] = ['; '.join(map(str, l)) for l in df['combinations']]

    # drop duplicates & remove the multi-index
    df = df.drop_duplicates()

    df = df.reset_index(level=1)

    # rejoin df_na to df now that combinations is a string column
    df = pd.concat([df, df_na], axis=0)

    # narrow down dataframe
    # get best combination (i.e. lowest number of elements)
    df['len'] = df['combinations'].str.count(';')

    # sort into top 5 options 
    df = df.sort_values('len', ascending=True).groupby(['weight']).head(5)
    df = df.sort_values(['weight','len']).drop(['len'], axis=1)

    # join back to original dataframe
    # recalculate the required weight to join back 
    final = df.rename(columns={'weight':'plate_weight','combinations':'plates' })
    # add bar weight and * 2
    final['weight'] = final['plate_weight'].apply(lambda x: (x*2)+bar)

    # merge onto program dataframe
    final = program_df.merge(final, on='weight', how='left')
    final.sort_values(['reps','%1RM','weight'], ascending=False)
    return final


# test the functions
program = create_program_df(program_data)
plates, plates_list = create_plates_df(plate_data)
final = generate_combinations(program, plates_list,bar)
final'''
print()

In [ ]:
# graveyard
'''
ORM = int(input("how much heavy shit can you lift? "))
count = int(input("How many sets?\n"))
reps = []
sets = []

for i in range(0, count):
    reps.append(int(input("Enter reps ")))
    sets.append(int(input("Enter %1RM ")))
    print('\n')
'''    
'''df = pd.DataFrame(list(map(lambda x,y:(x,y),reps,p1rm)), columns = ['reps', '%1RM'])
    df['lift'] = pd.Series([lift for x in range(len(df.index))])'''

'''
# test out a nested dictionary to create dataframes
d = {}
d['Bench'] = {}
d['Bench']['1RM'] = 40
d['Bench']['reps'] = [8, 5, 3, 2, 1]
d['Bench']['%1RM'] = [50, 60, 70, 80, 90]

d['Squat'] = {'1RM':60,'reps':[5, 5, 3],'%1RM':[60, 70, 80]}

df = pd.DataFrame(d)
df
'''
print()